## 1. SQLite with Python

In [2]:
import sqlite3
import pandas as pd
from contextlib import closing
import os

# Create in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create sample tables
cursor.execute('''
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name TEXT NOT NULL UNIQUE,
    category TEXT NOT NULL,
    price REAL NOT NULL,
    stock INTEGER NOT NULL
)
''')

cursor.execute('''
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL UNIQUE,
    email TEXT NOT NULL UNIQUE,
    created_at TEXT NOT NULL
)
''')

# Insert sample data
products = [
    ('Laptop', 'Electronics', 1200.00, 5),
    ('Mouse', 'Electronics', 25.00, 50),
    ('Desk', 'Furniture', 300.00, 3),
    ('Chair', 'Furniture', 150.00, 8)
]

cursor.executemany(
    'INSERT INTO products (product_name, category, price, stock) VALUES (?, ?, ?, ?)',
    products
)

users = [
    ('alice', 'alice@example.com', '2023-01-15'),
    ('bob', 'bob@example.com', '2023-02-20')
]

cursor.executemany(
    'INSERT INTO users (username, email, created_at) VALUES (?, ?, ?)',
    users
)
conn.commit()

print("✓ Database setup complete")
print(f"✓ Products table: {len(products)} items")
print(f"✓ Users table: {len(users)} users")

✓ Database setup complete
✓ Products table: 4 items
✓ Users table: 2 users


## Basic Query Execution

Execute SQL queries and fetch results using cursor and fetchall().

```python
cursor.execute('SELECT * FROM products')
results = cursor.fetchall()
for row in results:
    print(row)
```

In [3]:
# Fetch all results
cursor.execute('SELECT * FROM products')
all_products = cursor.fetchall()
print("All Products (cursor.fetchall()):")
for row in all_products:
    print(row)
print()

# Fetch one row
cursor.execute('SELECT * FROM users WHERE username = ?', ('alice',))
user = cursor.fetchone()
print("Specific User (cursor.fetchone()):")
print(user)
print()

# Fetch with column names
conn.row_factory = sqlite3.Row
cursor = conn.cursor()
cursor.execute('SELECT * FROM products WHERE category = ?', ('Electronics',))
electronics = cursor.fetchall()
print("Electronics with Column Names:")
for row in electronics:
    print(dict(row))

All Products (cursor.fetchall()):
(1, 'Laptop', 'Electronics', 1200.0, 5)
(2, 'Mouse', 'Electronics', 25.0, 50)
(3, 'Desk', 'Furniture', 300.0, 3)
(4, 'Chair', 'Furniture', 150.0, 8)

Specific User (cursor.fetchone()):
(1, 'alice', 'alice@example.com', '2023-01-15')

Electronics with Column Names:
{'product_id': 1, 'product_name': 'Laptop', 'category': 'Electronics', 'price': 1200.0, 'stock': 5}
{'product_id': 2, 'product_name': 'Mouse', 'category': 'Electronics', 'price': 25.0, 'stock': 50}


## Pandas Integration

Use Pandas read_sql_query() for convenient data analysis.

```python
df = pd.read_sql_query('SELECT * FROM products', conn)
print(df)  # Displays as formatted table
```

In [4]:
# Read SQL results into Pandas DataFrame
df_products = pd.read_sql_query('SELECT * FROM products', conn)
print("Products DataFrame:")
print(df_products)
print()

# Query with WHERE clause
df_expensive = pd.read_sql_query(
    'SELECT product_name, category, price FROM products WHERE price > 100 ORDER BY price DESC',
    conn
)
print("Expensive Products (>$100):")
print(df_expensive)
print()

# Use Pandas methods on DataFrame
print("Products DataFrame Info:")
print(f"Shape: {df_products.shape}")
print(f"Average Price: ${df_products['price'].mean():.2f}")
print(f"Total Stock: {df_products['stock'].sum()} units")

Products DataFrame:
   product_id product_name     category   price  stock
0           1       Laptop  Electronics  1200.0      5
1           2        Mouse  Electronics    25.0     50
2           3         Desk    Furniture   300.0      3
3           4        Chair    Furniture   150.0      8

Expensive Products (>$100):
  product_name     category   price
0       Laptop  Electronics  1200.0
1         Desk    Furniture   300.0
2        Chair    Furniture   150.0

Products DataFrame Info:
Shape: (4, 5)
Average Price: $418.75
Total Stock: 66 units


## Parameterized Queries (SQL Injection Prevention)

Use placeholders (?) to safely pass parameters.

```python
cursor.execute('SELECT * FROM products WHERE category = ?', ('Electronics',))
# NEVER: cursor.execute(f'SELECT * FROM products WHERE category = {category}')
```

In [5]:
# Safe parameterized query
category = 'Electronics'
price_threshold = 100

df = pd.read_sql_query(
    'SELECT * FROM products WHERE category = ? AND price >= ?',
    conn,
    params=(category, price_threshold)
)
print(f"Products in {category} category costing >= ${price_threshold}:")
print(df)
print()

# Multiple parameters
min_price, max_price = 20, 500
df = pd.read_sql_query(
    'SELECT product_name, price FROM products WHERE price BETWEEN ? AND ? ORDER BY price',
    conn,
    params=(min_price, max_price)
)
print(f"Products priced between ${min_price} and ${max_price}:")
print(df)

Products in Electronics category costing >= $100:
   product_id product_name     category   price  stock
0           1       Laptop  Electronics  1200.0      5

Products priced between $20 and $500:
  product_name  price
0        Mouse   25.0
1        Chair  150.0
2         Desk  300.0


## Error Handling

Handle database exceptions gracefully.

```python
try:
    cursor.execute('INSERT INTO products VALUES ...')
except sqlite3.IntegrityError as e:
    print(f"Integrity error: {e}")
except sqlite3.OperationalError as e:
    print(f"Operational error: {e}")
```

In [6]:
# Example 1: IntegrityError (duplicate key)
try:
    cursor.execute(
        'INSERT INTO products (product_name, category, price, stock) VALUES (?, ?, ?, ?)',
        ('Laptop', 'Electronics', 999.99, 10)
    )
conn.commit()
except sqlite3.IntegrityError as e:
    print(f"✗ Integrity Error: {e}")
    conn.rollback()
print()

# Example 2: OperationalError (table doesn't exist)
try:
    cursor.execute('SELECT * FROM nonexistent_table')
except sqlite3.OperationalError as e:
    print(f"✗ Operational Error: {e}")
print()

# Example 3: Successful insertion
try:
    cursor.execute(
        'INSERT INTO products (product_name, category, price, stock) VALUES (?, ?, ?, ?)',
        ('Monitor', 'Electronics', 400.00, 2)
    )
    conn.commit()
    print("✓ New product inserted successfully")
except sqlite3.Error as e:
    print(f"✗ Error: {e}")
    conn.rollback()

SyntaxError: expected 'except' or 'finally' block (2978540183.py, line 7)

## Context Managers for Resource Management

Use 'with' statement to automatically handle connection cleanup.

```python
with closing(sqlite3.connect(':memory:')) as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM products')
    # Connection automatically closes
```

In [ ]:
# Using context manager (proper resource management)
from contextlib import closing

print("Example 1: Context Manager with closing()")
with closing(sqlite3.connect(':memory:')) as test_conn:
    test_cursor = test_conn.cursor()
    test_cursor.execute('CREATE TABLE test (id INTEGER PRIMARY KEY, value TEXT)')
    test_cursor.execute('INSERT INTO test (value) VALUES ("Hello")')
    test_conn.commit()
    
    result = pd.read_sql_query('SELECT * FROM test', test_conn)
    print(result)
# Connection automatically closed here
print("✓ Connection closed automatically")
print()

print("Example 2: Manual resource management")
conn2 = sqlite3.connect(':memory:')
try:
    cursor2 = conn2.cursor()
    cursor2.execute('CREATE TABLE test2 (id INTEGER PRIMARY KEY)')
    cursor2.execute('INSERT INTO test2 VALUES (1)')
    conn2.commit()
    print("✓ Operations completed")
finally:
    conn2.close()
    print("✓ Connection closed in finally block")

## Writing DataFrames to Database

Use Pandas to_sql() method to write data to database.

```python
df = pd.DataFrame({'name': ['A', 'B'], 'value': [1, 2]})
df.to_sql('table_name', conn, if_exists='replace', index=False)
```

In [ ]:
# Create DataFrame and write to database
sales_data = {
    'product': ['Laptop', 'Mouse', 'Desk', 'Chair'],
    'quantity_sold': [3, 25, 1, 5],
    'revenue': [3600.00, 625.00, 300.00, 750.00]
}
df_sales = pd.DataFrame(sales_data)

print("DataFrame to write:")
print(df_sales)
print()

# Write to database
df_sales.to_sql('sales', conn, if_exists='replace', index=False)
print("✓ DataFrame written to 'sales' table")
print()

# Read it back to verify
df_verify = pd.read_sql_query('SELECT * FROM sales', conn)
print("Data read back from database:")
print(df_verify)
print()

# Calculate statistics
print("Sales Summary:")
print(f"Total Revenue: ${df_verify['revenue'].sum():.2f}")
print(f"Average Quantity Sold: {df_verify['quantity_sold'].mean():.1f} units")

## Database Drivers Comparison

| Library | Database | Type | Use Case |
|---------|----------|------|----------|
| **sqlite3** | SQLite | Built-in | Development, Testing, Single-file databases |
| **psycopg2** | PostgreSQL | 3rd-party | Production, Complex queries, Large databases |
| **pymysql** | MySQL/MariaDB | 3rd-party | Web applications, Shared hosting |
| **pyodbc** | SQL Server | 3rd-party | Enterprise environments, Windows |
| **cx_Oracle** | Oracle | 3rd-party | Enterprise databases, Large institutions |
| **sqlalchemy** | Multiple | ORM | Abstraction layer, Multiple database support |
| **pandas.sqlite3** | SQLite | Built-in | Data analysis, Quick prototyping |

## Best Practices
1. **Always use parameterized queries** to prevent SQL injection
2. **Handle exceptions** explicitly (IntegrityError, OperationalError)
3. **Use context managers** for automatic resource cleanup
4. **Commit or rollback** transactions appropriately
5. **Use Pandas** for data analysis and DataFrame operations
6. **Create indexes** on frequently searched columns
7. **Close connections** when done (or use context managers)

In [ ]:
# Final summary of all tables
print("=" * 60)
print("COMPLETE DATABASE SUMMARY")
print("=" * 60)
print()

print("PRODUCTS TABLE:")
df = pd.read_sql_query('SELECT * FROM products ORDER BY price DESC', conn)
print(df.to_string(index=False))
print()

print("USERS TABLE:")
df = pd.read_sql_query('SELECT * FROM users', conn)
print(df.to_string(index=False))
print()

print("SALES TABLE:")
df = pd.read_sql_query('SELECT * FROM sales ORDER BY revenue DESC', conn)
print(df.to_string(index=False))
print()

# Database statistics
print("DATABASE STATISTICS:")
stats = pd.read_sql_query(
    '''SELECT 
       (SELECT COUNT(*) FROM products) as total_products,
       (SELECT COUNT(*) FROM users) as total_users,
       (SELECT COUNT(*) FROM sales) as total_sales,
       ROUND((SELECT SUM(price * stock) FROM products), 2) as inventory_value''',
    conn
)
print(stats.to_string(index=False))

In [ ]:
import sqlite3
import pandas as pd
from contextlib import contextmanager

print("=" * 50)
print("PYTHON SQL INTEGRATION")
print("=" * 50)

print("\n" + "=" * 50)
print("1. SQLITE3 - BASIC OPERATIONS")
print("=" * 50)

# Create connection
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE products (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    price REAL,
    stock INTEGER
)
""")

# Insert data
products = [
    ('Laptop', 999.99, 5),
    ('Mouse', 29.99, 50),
    ('Keyboard', 79.99, 30),
    ('Monitor', 299.99, 10)
]

cursor.executemany("INSERT INTO products (name, price, stock) VALUES (?, ?, ?)", products)
conn.commit()

print("✓ Table created and data inserted")

# Query data
print("\nAll products:")
cursor.execute("SELECT * FROM products")
for row in cursor.fetchall():
    print(f"  {row}")

PYTHON SQL INTEGRATION

1. SQLITE3 - BASIC OPERATIONS
✓ Table created and data inserted

All products:
  (1, 'Laptop', 999.99, 5)
  (2, 'Mouse', 29.99, 50)
  (3, 'Keyboard', 79.99, 30)
  (4, 'Monitor', 299.99, 10)
